# Création d'une carte interactive

## Import des librairies

In [14]:
import geopandas as gpd
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from collections import Counter
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from pandas.core.arrays.sparse import SparseArray as _SparseArray

In [15]:
geolocator=Nominatim(user_agent="job_offers_map")

In [30]:
file_path='/Users/saimanenawal/Documents/offres_data1.csv'



In [31]:
data=pd.read_csv(file_path)

In [32]:
print(data.columns)

Index(['company', 'job_title', 'date', 'contract', 'location', 'salary',
       'remote', 'suitcase', 'description', 'lien', 'clock',
       'education_level'],
      dtype='object')


In [33]:
locations = data['location'].dropna().unique()

In [20]:
def get_coordinates(city):
    try:
        location = geolocator.geocode(city)
        if location: 
            return location.latitude, location.longitude
    except GeocoderTimedOut:
        return None
    return None

In [35]:
coordinates={}
for city in tqdm(locations, desc="Géocode des villes"):
    coords = get_coordinates(city)
    if coords:
        coordinates[city]= coords

Géocode des villes: 100%|██████████| 143/143 [03:25<00:00,  1.44s/it]


In [36]:
data['coordinates']=data['location'].map(coordinates)

In [37]:
france_map = folium.Map(location=[46.603354,1.888334], zoom_start=6)

In [38]:
for index, row in data.iterrows():
    if pd.notna(row['coordinates']):
        latitude,longitude = row['coordinates']
        folium.Marker(location=[latitude,longitude], popup=f"{row['job_title']}-{row['company']}",).add_to(france_map)

In [39]:
france_map.save("job_offers_map.html")

In [40]:
import webbrowser

In [41]:
webbrowser.open('job_offers_map.html')

True

In [42]:
france_map